# **Use keras to count Sea Lions**

implement @outrunner method

[Thanks to @outrunner(The 1st place winner)<br>for more information...][1]


  [1]: https://www.kaggle.com/c/noaa-fisheries-steller-sea-lion-population-count/discussion/35408

In [1]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import skimage.feature
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Conv2D, MaxPooling2D
import pandas as pd
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)

%matplotlib inline

Using TensorFlow backend.
/home/yanroo/anaconda3/envs/Keras/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/yanroo/anaconda3/envs/Keras/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/yanroo/anaconda3/envs/Keras/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/yanroo/anaconda3/

**Scale and patch**

In [2]:
r = 1     #scale down
width = 300 #patch size 
batch_size = 32

**Ignore Mismatched Images**

In [3]:
def read_ignore_list():
    df_ignore= pd.read_csv('./MismatchedTrainImages.txt')
    ignore_list= df_ignore['train_id'].tolist()
    
    return ignore_list

**Get dot coordinates and cut image to patches :** (thanks to Radu Stoicescu)

In [4]:
# def GetData(directory):
#     trainX = []
#     trainY = []
#     no_sealionX = []
#     no_sealionY = []
#     ignore_list = read_ignore_list()
#     img_count = 0
    
#     for i in range(10):
#         if i in ignore_list:
#             print(i)
#             continue
#         # read the Train and Train Dotted images
#         image_1 = cv2.imread("./TrainDotted/" + str(i)+ '.jpg')
#         image_2 = cv2.imread("./Train/" + str(i)+ '.jpg')
#         img1 = cv2.GaussianBlur(image_1,(5,5),0)
#         if image_1.shape != image_2.shape:
#             print(i)
#             image_2 = np.rot90(image_2, k=3)


#         # absolute difference between Train and Train Dotted
#         image_3 = cv2.absdiff(image_1,image_2)
#         mask_1 = cv2.cvtColor(image_1, cv2.COLOR_BGR2GRAY)
#         mask_1[mask_1 < 50] = 0
#         mask_1[mask_1 > 0] = 255
#         image_4 = cv2.bitwise_or(image_3, image_3, mask=mask_1)

#         # convert to grayscale to be accepted by skimage.feature.blob_log
#         image_6 = np.max(image_4,axis=2)

#         # detect blobs
#         blobs = skimage.feature.blob_log(image_6, min_sigma=3, max_sigma=7, num_sigma=1, threshold=0.05)

#         h,w,d = image_2.shape

#         res=np.zeros((int((w*r)//width)+1,int((h*r)//width)+1,5), dtype='int16')

#         for blob in blobs:
#             # get the coordinates for each blob
#             y, x, s = blob
#             # get the color of the pixel from Train Dotted in the center of the blob
#             b,g,R = img1[int(y)][int(x)][:]
#             x1 = int((x*r)//width)
#             y1 = int((y*r)//width)
#             # decision tree to pick the class of the blob by looking at the color in Train Dotted
#             if R > 225 and b < 25 and g < 25: # RED
#                 res[x1,y1,0]+=1
#             elif R > 225 and b > 225 and g < 25: # MAGENTA
#                 res[x1,y1,1]+=1
#             elif R < 75 and b < 50 and 150 < g < 200: # GREEN
#                 res[x1,y1,4]+=1
#             elif R < 75 and  150 < b < 200 and g < 75: # BLUE
#                 res[x1,y1,3]+=1
#             elif 60 < R < 120 and b < 50 and g < 75:  # BROWN
#                 res[x1,y1,2]+=1

#         ma = cv2.cvtColor((1*(np.sum(image_1, axis=2)>20)).astype('uint8'), cv2.COLOR_GRAY2BGR)
#         img = cv2.resize(image_2 * ma, (int(w*r),int(h*r)))
#         h1,w1,d = img.shape

#         for i in range(int(w1//width)):
#             for j in range(int(h1//width)):
#                 if np.sum(res[i,j,:]) >0:
#                     trainY.append(res[i,j,:])
#                     trainX.append(img[j*width:j*width+width,i*width:i*width+width,:])
#                     img_count += 1
#                 else:
#                     no_sealionY.append(res[i,j,:])
#                     no_sealionX.append(img[j*width:j*width+width,i*width:i*width+width,:])
        
#         if img_count >=5000:
#             return np.array(trainX), np.array(trainY), np.array(no_sealionX), np.array(no_sealionY)

#     return np.array(trainX), np.array(trainY), np.array(no_sealionX), np.array(no_sealionY)

# def rmse(predictions, targets):
#     return np.sqrt(((predictions - targets) ** 2).mean())

**Use only 1 image, split to train/test.**

In my real approach:

 - r = 1 to 0.6561 (0.9^0, 0.9^1 ... 0.9^4)
   
 - patch size = 300x300
   
 - cut whole training set to patches, number of positive(all) vs
   background(random) = 1 : 3
   
 - 95% for training, 5% for validation

 - data augmentation by flip, rotate, change saturation, brightness, contrast

In [5]:
# trainX, trainY, BGX, BGY = GetData("./Train")

# print(trainY.shape, trainY[0])

# np.random.seed(1004)
# randomize = np.arange(len(trainX))
# np.random.shuffle(randomize)

# trainX = trainX[randomize]
# trainY = trainY[randomize]

# n_train = int(len(trainX) * 0.95)
# valX = trainX[n_train:]
# valY = trainY[n_train:]
# trainX = trainX[:n_train]
# trainY = trainY[:n_train]



# randomize = np.arange(len(BGX))
# np.random.shuffle(randomize)

# BGX = BGX[randomize]
# BGY = BGY[randomize]

# trainX = np.concatenate((trainX, BGX[:9500]))
# trainY = np.concatenate((trainY, BGY[:9500]))
# valX = np.concatenate((valX, BGX[9500:10000]))
# valY = np.concatenate((valY, BGY[9500:10000]))

# # n_train = int(len(trainX) * 0.95)
# # valX = trainX[n_train:]
# # valY = trainY[n_train:]
# # trainX = trainX[:n_train]
# # trainY = trainY[:n_train]

# print(trainY.shape, trainY[0])
# print(valY.shape, valY[0])

In [6]:
# del BGX
# del BGY

**Augmentation**

In [7]:
from imgaug import augmenters as iaa

augmentation = iaa.Sequential([
    iaa.Fliplr(0.5), # horizontal flips
    iaa.Flipud(0.5), # vertical flips

    # Strengthen or weaken the contrast in each image.
    iaa.ContrastNormalization((0.75, 1.5)),
    #iaa.MultiplySaturation((0.5, 1.5)),
    iaa.Multiply((0.8, 1.2), per_channel=0),
    # Apply affine transformations to each image.
    # Scale/zoom them, translate/move them, rotate them and shear them.
    iaa.Affine(
      scale = [1, 0.9, 0.81, 0.73, 0.66],
        rotate=[90, 180, 270])
    ], random_order=True) # apply augmenters in random order

/home/yanroo/anaconda3/envs/Keras/lib/python3.6/site-packages/imgaug/imgaug.py:182: DeprecationWarning: Function `ContrastNormalization()` is deprecated. Use `imgaug.contrast.LinearContrast` instead.
  warn_deprecated(msg, stacklevel=3)


In [8]:
train_df = pd.read_csv('divide_image.csv')
val_df = pd.read_csv('val_image.csv')

In [9]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import preprocess_input, decode_predictions

# preprocessing image and divide validaiton set
train_datagen=ImageDataGenerator(preprocessing_function=augmentation.augment_image)
validation_datagen=ImageDataGenerator()


train_generator=train_datagen.flow_from_dataframe(train_df, 
                                                  directory='./',
                                                  x_col="filename",
                                                  y_col=["adult_males", "subadult_males", "adult_females", "juveniles", "pups"],               
                                                  target_size=(300, 300),
                                                  batch_size=batch_size,
                                                  class_mode='other',
                                                  shuffle=True)


validation_generator = validation_datagen.flow_from_dataframe(val_df,
                                               directory='./',
                                               x_col="filename",
                                               y_col=["adult_males", "subadult_males", "adult_females", "juveniles", "pups"],               
                                               target_size=(300, 300),
                                               batch_size=batch_size,
                                               class_mode='other')

Found 53442 validated image filenames.
Found 2815 validated image filenames.


In [10]:
# from keras.preprocessing.image import ImageDataGenerator
# from keras.applications.vgg16 import preprocess_input, decode_predictions

# # preprocessing image and divide validaiton set
# train_datagen=ImageDataGenerator(preprocessing_function=augmentation.augment_image)
# validation_datagen=ImageDataGenerator()

# train_generator=train_datagen.flow(trainX, 
#                                    trainY,
#                                    batch_size=batch_size,
#                                    shuffle=True)


# validation_generator = validation_datagen.flow(valX, 
#                                    valY,
#                                    batch_size=batch_size)

**Patches looks like :**

In [11]:
# fig = plt.figure(figsize=(12,12))
# for i in range(4):
#     ax = fig.add_subplot(1,4,i+1)
#     plt.imshow(cv2.cvtColor(trainX[i], cv2.COLOR_BGR2RGB))
# print(trainY[:4])

**Keras CNN model, for example**

In [12]:
from keras.applications.vgg16 import VGG16
from keras.layers import GlobalAveragePooling2D, Dense, Dropout
from keras.models import Model
from keras.layers.advanced_activations import LeakyReLU

# Transfer Learning from VGG16 architecture
# Without fully-connected layers
# define input shape as (224,224,3)
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(300,300,3))

# Define our own fully-connected layers & output layer
x = base_model.output
x = Flatten()(x)

x = Dense(1024, activation=LeakyReLU(alpha=0.1))(x)
# x = Dropout(0.3)(x)

preds = Dense(5, activation='linear')(x) #final layer with softmax activation



model = Model(inputs=base_model.input, outputs=preds)
model.summary()







Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 300, 300, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 300, 300, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 300, 300, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 150, 150, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 150, 150, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 150, 150, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 75, 75, 128)     

/home/yanroo/anaconda3/envs/Keras/lib/python3.6/site-packages/keras/activations.py:235: UserWarning: Do not pass a layer instance (such as LeakyReLU) as the activation argument of another layer. Instead, advanced activation layers should be used just like any other layer in a model.
  identifier=identifier.__class__.__name__))


In [13]:
for layer in model.layers[:-3]:
    layer.trainable = False
for layer in model.layers[-3:]:
    layer.trainable = True
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 300, 300, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 300, 300, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 300, 300, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 150, 150, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 150, 150, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 150, 150, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 75, 75, 128)       0   

full version model:

    initial_model = applications.VGG16(weights="imagenet", include_top=False, input_shape=(300,300,3))
    last = initial_model.output
    x = Flatten()(last)
    x = Dense(1024)(x)
    x = LeakyReLU(alpha=.1)(x)
    preds = Dense(5, activation='linear')(x)
    model = Model(initial_model.input, preds)

**Start training slowly :**

58 training epochs is a example in the kernel. I train the new FC several epochs, observe the loss, then train more layers until the train loss close to 0.3. The total epochs before training the whole network is about 60(I think it is too slow due to my poor skill). And the whole network takes 27 epochs to got 0.126/0.247 train/validation loss with private LB 11.7.
 - 60 epoch before whole layers

In [14]:
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint, TensorBoard


callbacks = [ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, verbose=1),
             EarlyStopping(monitor='val_loss', min_delta=0, patience=20, verbose=1),
             ModelCheckpoint('./logs/VGG16' + 'ep{epoch:03d}-loss{loss:.3f}-val_loss{val_loss:.3f}.h5', monitor='val_loss', save_weights_only=True, save_best_only=True, period=3, verbose=1),
             TensorBoard(log_dir='./logs/VGG16')]

In [15]:
optim = keras.optimizers.SGD(lr=1e-4, momentum=0.2)
model.compile(loss='mean_squared_error', optimizer=optim)


step_size_train = train_generator.n // train_generator.batch_size
step_size_val = validation_generator.n // validation_generator.batch_size

history = model.fit_generator(generator=train_generator, steps_per_epoch=step_size_train,
                              validation_data=validation_generator, validation_steps=step_size_val,
                              epochs=60, verbose=1, callbacks=callbacks)





Epoch 1/60
1670/1670 [==============================] - 325s 195ms/step - loss: 5.0541 - val_loss: 2.1588
Epoch 2/60
1670/1670 [==============================] - 325s 194ms/step - loss: 1.7151 - val_loss: 1.8245
Epoch 3/60
1670/1670 [==============================] - 328s 197ms/step - loss: 1.5746 - val_loss: 1.5274

Epoch 00003: val_loss improved from inf to 1.52739, saving model to ./logs/VGG16ep003-loss1.575-val_loss1.527.h5
Epoch 4/60
1670/1670 [==============================] - 323s 193ms/step - loss: 1.5071 - val_loss: 1.4413
Epoch 5/60
1670/1670 [==============================] - 320s 192ms/step - loss: 1.4276 - val_loss: 1.4425
Epoch 6/60
1670/1670 [==============================] - 311s 186ms/step - loss: 1.3744 - val_loss: 1.3819

Epoch 00006: val_loss improved from 1.52739 to 1.38186, saving model to ./logs/VGG16ep006-loss1.375-val_loss1.382.h5
Epoch 7/60
1670/1670 [==============================] - 320s 192ms/step - loss: 1.3582 - val_loss: 1.3203
Epoch 8/60
1670/1670 [=

1670/1670 [==============================] - 311s 186ms/step - loss: 1.0131 - val_loss: 1.0461
Epoch 57/60
1670/1670 [==============================] - 319s 191ms/step - loss: 1.0126 - val_loss: 1.1743

Epoch 00057: val_loss did not improve from 1.01579
Epoch 58/60
1670/1670 [==============================] - 319s 191ms/step - loss: 0.9978 - val_loss: 1.0480
Epoch 59/60
1670/1670 [==============================] - 315s 189ms/step - loss: 0.9939 - val_loss: 1.0409
Epoch 60/60
1670/1670 [==============================] - 316s 189ms/step - loss: 0.9997 - val_loss: 1.1297

Epoch 00060: ReduceLROnPlateau reducing learning rate to 9.999999974752428e-08.

Epoch 00060: val_loss did not improve from 1.01579
Epoch 00060: early stopping


**Then speed up :**

In [16]:
for layer in model.layers[:-7]:
    layer.trainable = False
for layer in model.layers[-7:]:
    layer.trainable = True
model.summary()

optim = keras.optimizers.SGD(lr=1e-4, momentum=0.9)
model.compile(loss='mean_squared_error', optimizer=optim)


history = model.fit_generator(generator=train_generator, steps_per_epoch=step_size_train,
                              validation_data=validation_generator, validation_steps=step_size_val,
                              initial_epoch=60, epochs=120, verbose=1, callbacks=callbacks)

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 300, 300, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 300, 300, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 300, 300, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 150, 150, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 150, 150, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 150, 150, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 75, 75, 128)       0   

/home/yanroo/anaconda3/envs/Keras/lib/python3.6/site-packages/keras/engine/training.py:493: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


Total params: 50,108,741
Trainable params: 42,473,477
Non-trainable params: 7,635,264
_________________________________________________________________
Epoch 61/120
1670/1670 [==============================] - 330s 198ms/step - loss: 1.3732 - val_loss: 0.8265
Epoch 62/120
1670/1670 [==============================] - 329s 197ms/step - loss: 1.0803 - val_loss: 0.7902
Epoch 63/120
1670/1670 [==============================] - 329s 197ms/step - loss: 0.9351 - val_loss: 0.9331

Epoch 00063: val_loss improved from 1.01579 to 0.93314, saving model to ./logs/VGG16ep063-loss0.935-val_loss0.933.h5
Epoch 64/120
1670/1670 [==============================] - 322s 193ms/step - loss: 0.8510 - val_loss: 0.7293
Epoch 65/120
1670/1670 [==============================] - 325s 194ms/step - loss: 0.8237 - val_loss: 0.6504
Epoch 66/120
1670/1670 [==============================] - 326s 195ms/step - loss: 0.8522 - val_loss: 0.7072

Epoch 00066: val_loss improved from 0.93314 to 0.70721, saving model to ./logs/VG

In [17]:
for layer in model.layers[:]:
    layer.trainable = True

model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 300, 300, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 300, 300, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 300, 300, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 150, 150, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 150, 150, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 150, 150, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 75, 75, 128)       0   

/home/yanroo/anaconda3/envs/Keras/lib/python3.6/site-packages/keras/engine/training.py:493: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


In [18]:
optim = keras.optimizers.SGD(lr=1e-5, momentum=0.9)
model.compile(loss='mean_squared_error', optimizer=optim)
# model.fit(trainX, trainY, epochs=30, verbose=2)
history = model.fit_generator(generator=train_generator, steps_per_epoch=step_size_train,
                              validation_data=validation_generator, validation_steps=step_size_val,
                              initial_epoch=120, epochs=220, verbose=1, callbacks=callbacks)

Epoch 121/220
1670/1670 [==============================] - 700s 419ms/step - loss: 0.6762 - val_loss: 0.5506
Epoch 122/220
1670/1670 [==============================] - 698s 418ms/step - loss: 0.4937 - val_loss: 0.4874

Epoch 00122: val_loss did not improve from 0.45152
Epoch 123/220
1670/1670 [==============================] - 698s 418ms/step - loss: 0.4640 - val_loss: 0.5248
Epoch 124/220
1670/1670 [==============================] - 701s 420ms/step - loss: 0.4416 - val_loss: 0.4774
Epoch 125/220
1670/1670 [==============================] - 702s 420ms/step - loss: 0.4275 - val_loss: 0.5070

Epoch 00125: val_loss did not improve from 0.45152
Epoch 126/220
1670/1670 [==============================] - 701s 420ms/step - loss: 0.4275 - val_loss: 0.4662
Epoch 127/220
1670/1670 [==============================] - 698s 418ms/step - loss: 0.4209 - val_loss: 0.5755
Epoch 128/220
1670/1670 [==============================] - 702s 420ms/step - loss: 0.4123 - val_loss: 0.4572

Epoch 00128: val_loss d

**Test :**

In [19]:
# from keras.models import load_model
# #model.load_weights('./logs/ep110-loss0.648-val_loss1.099.h5')

# result = model.predict(trainX)
# print('Training set --')
# print('    ground truth: ', np.sum(trainY, axis=0))
# print('  evaluate count: ', np.sum(result*(result>0.3), axis=0).astype('int'))

# result = model.predict(valX)
# print('Testing set --')
# print('    ground truth: ', np.sum(valY, axis=0))
# print('   predict count: ', np.sum(result*(result>0.3), axis=0).astype('int'))
# print(result.shape)

NameError: name 'trainX' is not defined

## Experience ##

The challenge is scale problem. They distinguish sea lion by size. In different images, one juveniles is larger than adult_females in another.

I can't handle it well, so I decided to fit LB score:

 - scale down testing image get better score
 - more juveniles (less adult_females) get better score

The final submission is made by:

 - testing image scale: 0.48
 - add 50% juveniles, and subtract adult_females with the same amount
 - add 20% pups

**Post processing details:**

These lucky variables are according to patch level regression.

The relationship between adult_females and juveniles in patches is:

![juveniles regression][1]

 - value in table = average of juveniles# / (adult_females# + juveniles#) @ juveniles number range in patches

 - r#.# means image scale

 - *#.# means juveniles increase ratio

  [1]: http://i.imgur.com/IkucSf6.gif